In [ ]:
import pandas as pd
import os
import pickle
import math
import numpy as np

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')   
os.chdir("/content/gdrive/My Drive/")

Mounted at /content/gdrive


In [ ]:
#files = os.listdir("HP comp")
BASE_DIR =  'MIE490: Capstone/Data/chronological/'
# files_to_process
os.listdir(BASE_DIR+'Male/')

In [ ]:
'''
Define the Athlete class to represent each athlete
'''
class Athlete:
    def __init__(self, fisCode, name, gender, country, beforeCompGlic, glic, initialRating, placement,confidence,volatility,birthdate, start_year, end_year, initial_level):
        self.fisCode = fisCode
        self.name = name
        self.gender = gender
        self.country = country
        self.beforeCompGlic = beforeCompGlic####
        self.glic = glic####
        self.initialRating = initialRating ####
        self.placement = placement #rank
        self.confidence = confidence ####
        self.volatility = volatility ####
        self.birthdate = birthdate
        self.start_year = start_year
        self.end_year = start_year
        self.initial_level = initial_level
        self.records = [None]*30
        

    def getInfo(self):
        return [self.fisCode, self.name, self.gender, self.birthdate, self.start_year, self.end_year, self.initial_level, self.country, self.glic]####

    def updatePlacement(self, newPlacement):
        self.placement = newPlacement

    def updateGlicko(self, year):
        self.beforeCompGlic = self.glic ####
        self.records[year-self.start_year] = self.glic
        self.end_year = year

    def __eq__(self, other):
        return self.fisCode == other.fisCode 

    def __hash__(self):
        return hash((self.name, self.fisCode))

    def __str__(self):
        return "{}: fisCode: {}, glicko: {}, gender: {}, country: {}".format(self.name, self.fisCode, self.glic, self.gender, self.country) ####


In [ ]:
'''
A class of competition representation. Process all participated athletes in a list
'''
class Competition:
    def __init__(self):
        self.competitors = []

    def addCompetitor(self, athlete):
        self.competitors.append(athlete)

In [ ]:
### GLOBAL VARIABLE
# initialize the dictionary to store all athletes data
athlete_dict = {}

# read the official ranking data
official_ranking_folder = 'MIE490: Capstone/Data/Ranking'
official_ranking_file_M = 'HP_M.xlsx'
official_ranking_file_W = 'HP_W.xlsx'
'''
xls_ranking_M = pd.ExcelFile(official_ranking_folder +'/' + official_ranking_file_M)
sheet_names_M = xls_ranking_M.sheet_names

xls_ranking_W = pd.ExcelFile(official_ranking_folder +'/' + official_ranking_file_W)
sheet_names_W = xls_ranking_W.sheet_names

'''

"\nxls_ranking_M = pd.ExcelFile(official_ranking_folder +'/' + official_ranking_file_M)\nsheet_names_M = xls_ranking_M.sheet_names\n\nxls_ranking_W = pd.ExcelFile(official_ranking_folder +'/' + official_ranking_file_W)\nsheet_names_W = xls_ranking_W.sheet_names\n\n"

In [ ]:
'''
Process one competition data into a competition class object. 
Each athlete is updated with his/her rank in this competition.
After processing all athletes, their competition ranks  will be used to calculate Elo score. 
'''

def competitionFileParser(df, comp_year):
    # df is the dataframe that contains competition data for one competition
    # date = df.loc[0,'Race Date1']
    gender = df['Gender (Races!Sb)'].values[0]
    comp_level = df['Cup Type'].values[0]
    competition = Competition()
    for row in (df.index):
        fisCode = df.loc[row,'Fis Code']
        placement = df.loc[row,'Rank']
        birthdate = df.loc[row, 'Birthdate']
        # Initialize a new Athlete object
        if fisCode not in athlete_dict.keys():
            # NOTE: some athlete has no name...
            name = df.loc[row,'Athlete Name']
            country = df.loc[row,'Nation']
            # placement = df.loc[row,'Rank']
            #### initial rating by competition
            if comp_level == 'WSC' or comp_level == 'WC':
                # Championship
                glic = 1500
            elif comp_level == 'NAC' or comp_level == 'WJC':
                glic = 1300
            elif comp_level == 'NC' or comp_level == 'NC/FI':
                glic = 1000
            else:
                glic = 1300
                print(comp_level)
                print(df['Race Date'].values[0])
            #### initial value according to the sample
            athlete = Athlete(fisCode, name, gender, country, glic, glic, glic, placement,_INITCONF,_INITVOL, birthdate, comp_year, comp_year, comp_level) ####
            athlete_dict[fisCode] = athlete
        # update athlete's rank in current competition
        else:
            athlete = athlete_dict.get(fisCode)
            athlete.updatePlacement(placement)

        # add athlete to the current competition class
        competition.addCompetitor(athlete)
        
    return competition


'''
Use the training data prior 2013-12-05 to calculate the new scores for all the 
athletes.

folderName: The path to either Male or Female competition data
'''

def train_system(folderName, end_date):
    files = os.listdir(folderName)
    # sort files in reverse chronological order 
    files.sort(reverse = False)
    
    # Go through each competition data
    for file in files:
        # discard qualification competition
        # if file == 'M_HP_QL.xlsx':
        #      continue
        end_year = int(str(end_date)[:4])
        file_year = int(file.split('_')[-1][:4])
        if file_year > end_year:
            continue 
        if 'xlsx' in file and '.DS_Store' not in file and '~$' not in file:
            xls = pd.ExcelFile(folderName +'/' + file)
            sheet_names = xls.sheet_names
            ## each sheet is a competition in that competition level
            for sheet in sheet_names:
                comp_date = int(sheet.split('_')[-1])
                comp_year = int(sheet.split('_')[-1][0:4])
                # ground truth ranking data starts from 20131205
                # use all data prior 20131205 as training data
                if comp_date < end_date:
                  df = xls.parse(sheet,header = 0)
                  # remove the athlete who DNS
                  df = df[df['Status']== 'QLF']
                  # process competition data 
                  competition = competitionFileParser(df, comp_year)
                  # run elo calculation for this competition
                  calculateGlicko(competition.competitors, comp_year) ####
                  
            

In [ ]:
'''
Sort the athlete dictionary and return a DataFrame with sorted athlete information
'''
def sort_athlete(athlete_dict):
    sorted_athlete = sorted(athlete_dict.items(), key=lambda x: x[1].glic, reverse=True) ####

    temp = []
    for fiscode, athlete in sorted_athlete:
      temp.append(athlete.getInfo())

    return pd.DataFrame(temp, columns = ['fiscode', 'name', 'gender', 'birthdate','First competition year','Last competition year', 'First competition level', 'country', 'glicko'])####

In [ ]:
'''
Search for the ranking data that is the most recent to the given comp_date.
The ranking data comes from the given official athlete ranking list.

Return the ranking data that is closest to the competition date.

This ranking data will be used as benchmark for comparsion to our point system

sheet_names: a list of sheet names represents the ranking data. In order of most recent ranking date, that is most recent to oldest
'''
def search_official_ranking(xls, sheet_names, comp_date):
    for sheet in sheet_names:
        sheet_date = sheet.split('_')[-1]
        # found the ranking data prior to the competition date
        if comp_date > sheet_date:
            return xls.parse(sheet,header = 0)


In [ ]:
# Glicko v2
# Background information - https://en.wikipedia.org/wiki/Glicko_rating_system
# Based on this equation - http://www.glicko.net/glicko/glicko2.pdf

#Glicko's principal contribution to measurement is "ratings reliability", called RD, for ratings deviation.

_MAXSIZE = 186
_MAXMULTI = .272
_MULTISLOPE = .00391
_WIN = 1.0
_LOSS = 0
_CATCH = .5
_INITCONF = 100 ####
_INITVOL = .001 ####
_VOL = .05
_CONV = 173.7178
_EPS = 0.001


def findSigma(mu, phi, sigma, change, v):
    alpha = math.log(sigma ** 2)

    def f(x):
        tmp = phi ** 2 + v + math.exp(x)
        a = math.exp(x) * (change ** 2 - tmp) / (2 * tmp ** 2)
        b = (x - alpha) / (_VOL ** 2)
        return a - b

    a = alpha
    if change ** 2 > phi ** 2 + v:
        b = math.log(change ** 2 - phi ** 2 - v)
    else:
        k = 1
        while f(alpha - k * _VOL) < 0:
            k += 1
        b = alpha - k * _VOL
    fa = f(a)
    fb = f(b)
    # Larger _EPS used to speed iterations up slightly
    while abs(b - a) > _EPS:
        c = a + (a - b) * fa / (fb - fa)
        fc = f(c)
        if fc * fb < 0:
            a = b
            fa = fb
        else:
            fa /= 2
        b = c
        fb = fc
    return math.e ** (a / 2)

def calculateGlicko(players, comp_year):
    N = len(players)
    if N > _MAXSIZE:
        multi = _MAXMULTI
    else:
        multi = _WIN - _MULTISLOPE * N
    # compare every head to head matchup in a given compeition
    for i in range(0, N-1):
        player1 = players[i]
        mu = (player1.beforeCompGlic - player1.initialRating ) / _CONV #_INITRAT
        phi = player1.confidence / _CONV
        sigma = player1.volatility
        v_inv = 0
        delta = 0
        for j in range(i+1, N):
            player2 = players[j]
            if i is not j:
                oppMu = (player2.beforeCompGlic - player2.initialRating) / _CONV #_INITRAT
                oppPhi = player2.confidence / _CONV
                if player1.placement > player2.placement:
                    S = _LOSS
                elif player1.placement < player2.placement:
                    S = _WIN
                else:
                    S = _CATCH
                # Change the weight of the matchup based on opponent confidence
                weighted = 1 / math.sqrt(1 + 3 * oppPhi ** 2 / math.pi ** 2)
                # Change the weight of the matchup based on competition size
                weighted = weighted * multi
                expected_score = 1 / (1 + math.exp(-weighted * (mu - oppMu)))
                v_inv += weighted ** 2 * expected_score * \
                    (1 - expected_score)
                delta += weighted * (S - expected_score)
        if v_inv != 0:
            v = 1 / v_inv
            change = v * delta
            newSigma = findSigma(mu, phi, sigma, change, v)
            phiAst = math.sqrt(phi ** 2 + newSigma ** 2)
            # New confidence based on competitors volatility and v
            newPhi = 1 / math.sqrt(1 / phiAst ** 2 + 1 / v)
            newMu = mu + newPhi ** 2 * delta
            player1.glic = newMu * _CONV +  player1.initialRating #_INITRAT
            player1.confidence = newPhi * _CONV
            player1.volatility = newSigma
    for player in players:
        player.updateGlicko(comp_year) ####
        

In [ ]:
'''
Evaluate the system starting from the start_date to validation_end_date.
After evaluate athletes basen on their current competition results and prior Elo scores,
the ranking system update elo scores for all the athletes to fully reflect their true skill based on thier performance.

start_date should be at least > 20131205 because the official published ranking data is only availble after this time

xls_ranking and ranking_sheet_names help the system to find the most recent published offcial ranking
The offcial ranking is used to evaluate the accuracy of the ranking system.

''' 
def evaluate_system(folderName, start_date, validation_end_date, xls_ranking, ranking_sheet_names, show_yearly = False):
    files = os.listdir(folderName)
    # sort files in reverse chronological order 
    files.sort(reverse = False)
    # Go through each competition data
    validation_result = pd.DataFrame(columns=['competition', 'Glicko accuracy', 'Official accuracy','Absolute Glicko Dev', 'Absolute Official Dev','Pairwise Glicko Dev', 'Pairwise Official Dev'])
    testing_result = pd.DataFrame(columns=['competition', 'Glicko accuracy', 'Official accuracy','Absolute Glicko Dev', 'Absolute Official Dev', 'Pairwise Glicko Dev', 'Pairwise Official Dev'])
    start_year = int(str(start_date)[:4])
    for file in files:
        start_year = int(str(start_date)[:4])
        file_year = int(file.split('_')[-1][:4])
        if file_year < start_year:
            continue
        print('Currently processing file: {}\n'.format(file))
        yearly_result = pd.DataFrame(columns=['competition', 'Glicko accuracy', 'Official accuracy','Absolute Glicko Dev', 'Absolute Official Dev', 'Pairwise Glicko Dev', 'Pairwise Official Dev']) ####
        if 'xlsx' in file and '.DS_Store' not in file and '~$' not in file:
            xls = pd.ExcelFile(folderName +'/' + file)
            sheet_names = xls.sheet_names
            #sheet_names.sort(reverse = False)
            ## each sheet is a competition in that competition level
            for sheet in sheet_names:
                comp_date = int(sheet.split('_')[-1])
                comp_year = int(sheet.split('_')[-1][0:4])
                # Note: ground truth ranking data starts from 20131205
                # Evaluate the system starting from the start_date
                if comp_date > start_date:
                  df = xls.parse(sheet,header = 0)
                  # remove the athletes who have not finished the competition
                  df = df[df['Status']== 'QLF']
                  # get the official ranking data
                  official_ranking = search_official_ranking(xls_ranking, ranking_sheet_names, sheet.split('_')[-1])####
                  # feed the competition data and official ranking to compute the accuracy
                  glicko_accuracy, official_accuracy, glicko_deviation, official_deviation, glicko_pairwise_deviation, official_pairwise_deviation = evaluation(df, official_ranking) ####
                  #print('elo: {}, official: {}'.format(elo_accuracy, official_accuracy))
                  yearly_result = yearly_result.append({'competition':sheet,
                                          'Glicko accuracy': glicko_accuracy,
                                          'Official accuracy': official_accuracy, 
                                          'Absolute Glicko Dev': glicko_deviation,
                                          'Absolute Official Dev': official_deviation,
                                          'Pairwise Glicko Dev': glicko_pairwise_deviation, 
                                          'Pairwise Official Dev': official_pairwise_deviation}, ignore_index=True)####
                  if comp_date <= validation_end_date:
                      validation_result = validation_result.append({'competition':sheet,
                                          'Glicko accuracy': glicko_accuracy,
                                          'Official accuracy': official_accuracy, 
                                          'Absolute Glicko Dev': glicko_deviation,
                                          'Absolute Official Dev': official_deviation,
                                          'Pairwise Glicko Dev': glicko_pairwise_deviation, 
                                          'Pairwise Official Dev': official_pairwise_deviation}, ignore_index=True)
                  else:
                      testing_result = testing_result.append({'competition':sheet,
                                          'Glicko accuracy': glicko_accuracy,
                                          'Official accuracy': official_accuracy, 
                                          'Absolute Glicko Dev': glicko_deviation,
                                          'Absolute Official Dev': official_deviation,
                                          'Pairwise Glicko Dev': glicko_pairwise_deviation, 
                                          'Pairwise Official Dev': official_pairwise_deviation}, ignore_index=True) 
                  ##### After evaluate the competition, re-calculate participated athletes' Glicko
                  # process competition data 
                  competition = competitionFileParser(df, comp_year)
                  # run Glicko calculation for this competition
                  calculateGlicko(competition.competitors, comp_year)####
            if show_yearly:
                print(yearly_result.mean())
                print('\n')

    print("----------------------------------------------------------------------")
    # here prints out the summarized results for validation and testing periods respectively
    print('Validation results ({} - {})\n'.format(start_date, validation_end_date))
    print(validation_result.mean())
    #print(validation_result)
    print('\n')
    print('Testing results: \n')
    print(testing_result.mean())
    #print(testing_result)
    print('\n')
    return validation_result.mean()

In [ ]:
def evaluation(df, official_ranking):
    # get all the athletes who participated in the current competition
    participated_athlete = {}
    participated_official_ranking = pd.DataFrame(columns=['fiscode', 'name', 'points'])

    for fis in df.loc[:,'Fis Code']:
        # get from Glicko system
        existed_athlete = athlete_dict.get(fis)
        # check if this athlete participated in previous competitions
        # only compare athletes who have competed before
        if existed_athlete:
          participated_athlete[fis] = existed_athlete
          
          # get the athlete data from the offcial ranking data
          official_ranked_athlete = official_ranking[official_ranking['fis code'] == fis]
          # this athlete has not competed in the most recent 52 weeks, which means not on the official ranking. 
          # set Default point to ZERO
          if official_ranked_athlete.empty:
              participated_official_ranking = participated_official_ranking.append({'fiscode': fis,
                                                                                    'name': existed_athlete.name,
                                                                                    'points':-1}, ignore_index=True)
          else:
              participated_official_ranking = participated_official_ranking.append({'fiscode': fis,
                                                                                    'name': existed_athlete.name,
                                                                                    'points':official_ranked_athlete['points'].values[0]}, ignore_index=True)    
              
    #sort the official ranking results
    participated_official_ranking = participated_official_ranking.sort_values(by=['points'],ascending=False, ignore_index=True)

    # sort the glicko point system
    participated_glicko_ranking = sort_athlete(participated_athlete) ####

    # filter the athletes who have participated in previous competitions
    df_filtered = df[df['Fis Code'].isin(participated_glicko_ranking['fiscode'])].loc[:,['Athlete Name', 'Fis Code', 'Rank', 'Race Points']] ####

    # pairwise compare every two athletes
    glicko_accuracy, official_accuracy = pairwise_accuracy(df_filtered, participated_glicko_ranking, participated_official_ranking)
    glicko_deviation, official_deviation = deviation_accuracy(df_filtered, participated_glicko_ranking, participated_official_ranking)
    glicko_pairwise_deviation, official_pairwise_deviation = pairwise_deviation_accuracy(df_filtered, participated_glicko_ranking, participated_official_ranking)
    
    return glicko_accuracy, official_accuracy, glicko_deviation, official_deviation, glicko_pairwise_deviation, official_pairwise_deviation


In [ ]:
'''
pairwise compare athletes using Elo ranking and the official ranking with the actual competition ranks
This method evaluates whether one athlete ranks higher than others according to the predicted ranks from Elo and Official data, and compare this prediction to the actual competition result
'''
def pairwise_accuracy(competition_ranking_df, glicko_ranking_df, official_ranking_df):    
    count = 0
    glicko_correct = 0 ####
    official_correct = 0
    # Note: some athletes got filtered out b/c they didn't compete before or DNS; 
    # So i used index lookup. Index 1 is 'Fis Code'
    for i in range(competition_ranking_df.shape[0] - 1):
        athlete1 = competition_ranking_df.iloc[i, 1]
        athlete1_glic = glicko_ranking_df[glicko_ranking_df['fiscode']==athlete1]['glicko'].values[0]
        athlete1_official = official_ranking_df[official_ranking_df['fiscode']==athlete1]['points'].values[0]
        for j in range(i+1, competition_ranking_df.shape[0]):
            athlete2 = competition_ranking_df.iloc[j, 1]
            athlete2_glic = glicko_ranking_df[glicko_ranking_df['fiscode']==athlete2]['glicko'].values[0]
            athlete2_official = official_ranking_df[official_ranking_df['fiscode']==athlete2]['points'].values[0]
            if athlete1_glic > athlete2_glic and athlete2_glic != 0:
                glicko_correct += 1
            if athlete1_official > athlete2_official and athlete2_official != -1:
                official_correct += 1
            count += 1
    return glicko_correct/count, official_correct/count

In [ ]:
'''
Calculate accuracy by comparing the deviation of the competition rank positions from Elo system and Official rankings to the true competition ranks.
This method evaluates one athlete's predicted rank position his/her true ranks based on Elo and Official data

'''
def deviation_accuracy(competition_ranking_df, glicko_ranking_df, official_ranking_df):    
    count = 0
    glicko_deviation = 0
    official_deviation = 0
    # Note: some athletes got filtered out b/c they didn't compete before or DNS; 
    # So i used index lookup. Index 1 is 'Fis Code'
    for i in range(competition_ranking_df.shape[0]):
        athlete1 = competition_ranking_df.iloc[i, 1]
        athlete1_glic_rank = glicko_ranking_df[glicko_ranking_df['fiscode']==athlete1].index[0]
        athlete1_official_rank = official_ranking_df[official_ranking_df['fiscode']==athlete1].index[0]
          
        glicko_deviation += abs(i - athlete1_glic_rank)
        official_deviation += abs(i - athlete1_official_rank)
        count += 1
    return glicko_deviation/count, official_deviation/count

In [ ]:
'''
Calculate accuracy by comparing the pairwise deviation of the rank positions from Elo, Official rankings to the true competition ranks.
This method evaluates the difference between one athlete's predicted rank position to other athletes' and one's true rank to others based on Elo and Official data

'''
def pairwise_deviation_accuracy(competition_ranking_df, elo_ranking_df, official_ranking_df):    
    count = 0
    elo_pairwise_deviation = 0
    official_pairwise_deviation = 0
    # Note: some athletes got filtered out b/c they didn't compete before or DNS; 
    # So i used index lookup. Index 1 is 'Fis Code'
    for i in range(competition_ranking_df.shape[0] - 1):
        athlete1 = competition_ranking_df.iloc[i, 1]
        athlete1_elo_rank = elo_ranking_df[elo_ranking_df['fiscode']==athlete1].index[0]
        athlete1_official_rank = official_ranking_df[official_ranking_df['fiscode']==athlete1].index[0]
        for j in range(i+1, competition_ranking_df.shape[0]):
            athlete2 = competition_ranking_df.iloc[j, 1]
            athlete2_elo_rank = elo_ranking_df[elo_ranking_df['fiscode']==athlete2].index[0]
            athlete2_official_rank = official_ranking_df[official_ranking_df['fiscode']==athlete2].index[0]
            true_diff = i - j
            elo_pairwise_deviation += abs(true_diff - (athlete1_elo_rank - athlete2_elo_rank))
            official_pairwise_deviation += abs(true_diff - (athlete1_official_rank - athlete2_official_rank))
            #print("glicko paiwise dev: ", abs(true_diff - (athlete1_elo_rank - athlete2_elo_rank)), "official_pairwise dev: ", abs(true_diff - (athlete1_official_rank - athlete2_official_rank)))
            count += 1
    return elo_pairwise_deviation/count, official_pairwise_deviation/count

In [ ]:
# train the our point system up until the end_date
official_ranking_file = 'HP_M.xlsx'
xls_ranking_M = pd.ExcelFile(official_ranking_folder +'/' + official_ranking_file)
sheet_names_M = xls_ranking_M.sheet_names
athlete_dict = {}
train_system(BASE_DIR+'Male', end_date = 20131231)
evaluate_system(BASE_DIR+'Male', start_date = 20140101, validation_end_date = 20171231, xls_ranking = xls_ranking_M, ranking_sheet_names=sheet_names_M, show_yearly=True)


In [ ]:
# train the our point system up until the end_date
official_ranking_file = 'HP_W.xlsx'
xls_ranking_W = pd.ExcelFile(official_ranking_folder +'/' + official_ranking_file)
sheet_names_W = xls_ranking_W.sheet_names
athlete_dict = {}
train_system(BASE_DIR+'Female', end_date = 20131231)
evaluate_system(BASE_DIR+'Female', start_date = 20140101, validation_end_date = 20171231, xls_ranking = xls_ranking_W, ranking_sheet_names=sheet_names_W, show_yearly=True)

## Hyperparameter tuning

In [ ]:
### parameters Tuning ###
gender = 'Female'
# read the official ranking data
official_ranking_folder = 'MIE490: Capstone/Data/Ranking'
if gender == 'Male':
  official_ranking_file = 'HP_M.xlsx'
else:
  official_ranking_file = 'HP_W.xlsx'
xls_ranking = pd.ExcelFile(official_ranking_folder +'/' + official_ranking_file)
sheet_names = xls_ranking.sheet_names

count = 1
_MAXSIZE = 186
_MAXMULTI = .272
_MULTISLOPE = .00391
_WIN = 1.0
_LOSS = 0
_CATCH = .5
_INITCONF_list = [30,40,50,60,70,80,90,100] #350
_INITVOL_list = [0.008,0.007,0.006,0.005,0.004,0.003,0.002,0.001] #0.05
_VOL = 0.05 #0.5
_CONV = 173.7178
_EPS = 0.001
best = 0
for conf ,ivol in zip(_INITCONF_list,_INITVOL_list):
    _INITCONF = conf
    _INITVOL = ivol
    print(count)
    print("------------------------- \n", "_INITCONF = ", conf, "_INITVOL = ",ivol)
    athlete_dict = {}
    train_system(BASE_DIR+gender, end_date = 20131231)
    validation_result = evaluate_system(BASE_DIR+gender, start_date = 20140101, validation_end_date = 20171231, xls_ranking = xls_ranking, ranking_sheet_names=sheet_names, show_yearly=False)
    count +=1
    if validation_result['Glicko accuracy'] > best:
      best = validation_result['Glicko accuracy']
      print("best _INITCONF = ", conf, "_INITVOL = ",ivol)
      print("accuracy: ",validation_result['Glicko accuracy'])
